+ 1. Определить бизнес-цель.
+ 2. Определить цель технического проекта.
+ 3. При необходимости следует использовать методы сбора и подготовки данных, которые были рассмотрены в лекции № 2.
+ 4. Выполнить разбиение набора данных на обучающую, контрольную (при необходимости) и тестовую выборки для устранения проблемы просачивания данных.
+- 5. Оценить сбалансированность выборок для набора данных. Применить методы приращения (аугментации) данных при необходимости.
+ 6. Выполнить процесс конструирования признаков для решения задачи. Задача определяется совокупностью бизнес-цели и цели технического проекта.
7. При конструировании признаков обязательно необходимо выполнить: унитарное кодирование категориальных признаков (one-hot encoding), дискретизацию числовых признаков, «ручной» синтез признаков, масштабирование признаков на основе нормировки и стандартизации.
8. При необходимости выполнить конструирование признаков с применением фреймворка Featuretools (https://featuretools.alteryx.com/en/stable/) (Только для ИТ).
9. Оценить качество полученных признаков по следующим критериям: предсказательная способность, скорость вычисления, надежность, корреляция и цельность.

In [80]:
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

df = pd.read_excel("data/India_tourist_places.xlsx")

# df.info()
display(df.Type.value_counts())
display(df.head())

df = df.drop(["WeeklyOff", "EstablishmentYear"], axis=1)


Type
Temple           57
Beach            25
Fort             21
Lake             15
National Park    14
                 ..
Village           1
Island            1
Township          1
Orchard           1
Gravity Hill      1
Name: count, Length: 75, dtype: int64

,Zone,State,City,Name,Type,EstablishmentYear,TimeNeededToVisitInHrs,GoogleReviewRating,EntranceFeeInINR,AirportWith50kmRadius,WeeklyOff,Significance,DSLRAllowed,NumberOfGoogleReviewInLakhs,BestTimeToVisit,ImageURL
0,Western,Gujarat,Rann of Kutch,Rann Utsav,Cultural,Unknown,3.0,4.9,7500,Yes,NaN,Cultural,Yes,0.10,Evening,https://www.rannutsav.com/blog/wp-content/uplo...
1,Northern,Punjab,Amritsar,Golden Temple (Harmandir Sahib),Religious Site,1604,1.5,4.9,0,Yes,NaN,Spiritual,Yes,1.90,All,https://upload.wikimedia.org/wikipedia/commons...
2,Northern,Ladakh,Leh,Pangong Tso,Lake,Unknown,2.0,4.9,20,Yes,NaN,Nature,Yes,0.15,Morning,https://dynamic-media-cdn.tripadvisor.com/medi...
3,Western,Maharastra,Mumbai,Siddhivinayak Temple,Temple,1881,2.0,4.8,0,Yes,NaN,Religious,No,1.05,All,https://upload.wikimedia.org/wikipedia/commons...
4,Western,Gujarat,Somnath,Somnath Temple,Temple,1951,2.0,4.8,0,No,NaN,Religious,No,0.39,Morning,https://upload.wikimedia.org/wikipedia/commons...


In [81]:
display(df.isnull().sum())
display()
display(df.isnull().any())
display()

# for i in df.columns:
#     null_rate = df[i].isnull().sum() / len(df) * 100
#     if null_rate > 0:
#         display(f"{i} процент пустых значений: %{null_rate:.2f}")

# df = df.drop(["WeeklyOff", "Name", "EstablishmentYear"], axis=1)

# display(df.head())

Zone                           0
State                          0
City                           0
Name                           0
Type                           0
TimeNeededToVisitInHrs         0
GoogleReviewRating             0
EntranceFeeInINR               0
AirportWith50kmRadius          0
Significance                   0
DSLRAllowed                    0
NumberOfGoogleReviewInLakhs    0
BestTimeToVisit                0
ImageURL                       0
dtype: int64

Zone                           False
State                          False
City                           False
Name                           False
Type                           False
TimeNeededToVisitInHrs         False
GoogleReviewRating             False
EntranceFeeInINR               False
AirportWith50kmRadius          False
Significance                   False
DSLRAllowed                    False
NumberOfGoogleReviewInLakhs    False
BestTimeToVisit                False
ImageURL                       False
dtype: bool

In [82]:
# Унитарное кодирование категориальных признаков (one-hot encoding)

encoder = OneHotEncoder(sparse_output=False, drop="first")

features_to_encode = ["Zone", "State", "City", "Type", "TimeNeededToVisitInHrs", 
        "GoogleReviewRating", "EntranceFeeInINR", "AirportWith50kmRadius", 
        "Significance", "NumberOfGoogleReviewInLakhs"]

encoded_values = encoder.fit_transform(df[features_to_encode])
encoded_columns = encoder.get_feature_names_out(features_to_encode)
encoded_values_df = pd.DataFrame(encoded_values, columns=encoded_columns)
df = pd.concat([df, encoded_values_df], axis=1)
df

,Zone,State,City,Name,Type,TimeNeededToVisitInHrs,GoogleReviewRating,EntranceFeeInINR,AirportWith50kmRadius,Significance,...,NumberOfGoogleReviewInLakhs_1.6,NumberOfGoogleReviewInLakhs_1.8,NumberOfGoogleReviewInLakhs_1.9,NumberOfGoogleReviewInLakhs_2.0,NumberOfGoogleReviewInLakhs_2.1,NumberOfGoogleReviewInLakhs_2.2,NumberOfGoogleReviewInLakhs_2.25,NumberOfGoogleReviewInLakhs_2.5,NumberOfGoogleReviewInLakhs_2.6,NumberOfGoogleReviewInLakhs_3.6
0,Western,Gujarat,Rann of Kutch,Rann Utsav,Cultural,3.0,4.9,7500,Yes,Cultural,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Northern,Punjab,Amritsar,Golden Temple (Harmandir Sahib),Religious Site,1.5,4.9,0,Yes,Spiritual,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Northern,Ladakh,Leh,Pangong Tso,Lake,2.0,4.9,20,Yes,Nature,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Western,Maharastra,Mumbai,Siddhivinayak Temple,Temple,2.0,4.8,0,Yes,Religious,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Western,Gujarat,Somnath,Somnath Temple,Temple,2.0,4.8,0,No,Religious,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,Southern,Tamil Nadu,Kodaikanal,Kodaikanal Lake,Lake,2.0,3.9,0,No,Recreational,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298,Southern,Kerala,Kumarakom,Kumarakom Bird Sanctuary,Bird Sanctuary,2.0,3.8,50,Yes,Wildlife,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299,Northern,Himachal Pradesh,Kufri,Kufri Fun World,Ski Resort,5.0,3.8,1500,Yes,Recreational,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
300,Northern,Ladakh,Leh,Magnetic Hill,Gravity Hill,0.5,3.7,0,Yes,Nature,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
# Дискретизация числовых признаков

labels = ["free", "cheap", "average", "expensive"]
hist3, bins3 = np.histogram(
    df["EntranceFeeInINR"].fillna(df["EntranceFeeInINR"].median()), bins=[-1, 1, 2500, 5000, 7501]
)
# bins3, hist3

pd.concat([df["EntranceFeeInINR"], pd.cut(df["EntranceFeeInINR"], list(bins3))],axis=1).head(5)
pd.concat([df["EntranceFeeInINR"], pd.cut(df["EntranceFeeInINR"], list(bins3), labels=labels)], axis=1).head(20)

# labels = ["free", "cheap", "average", "expensive"]
# num_bins = 4

# # EntranceFeeInINR

# hist1, bins1 = np.histogram(df["EntranceFeeInINR"].fillna(df["EntranceFeeInINR"].median()), bins=num_bins)
# bins1, hist1


,EntranceFeeInINR,EntranceFeeInINR
0,7500,expensive
1,0,free
2,20,cheap
3,0,free
4,0,free
5,0,free
6,0,free
7,0,free
8,0,free
9,10,cheap


In [91]:
# «ручной» синтез признаков

df_norm = df.copy()

df_norm["Best"] = np.where(df_norm["GoogleReviewRating"] >= 4.5, "Yes", "No")

display(df_norm[df_norm["Best"] == "No"][["Name", "GoogleReviewRating", "Best"]])

,Name,GoogleReviewRating,Best
196,National Science Centre,4.4,No
197,Haji Ali Dargah,4.4,No
198,Lalbagh Botanical Garden,4.4,No
199,Cubbon Park,4.4,No
200,Golconda Fort,4.4,No
...,...,...,...
297,Kodaikanal Lake,3.9,No
298,Kumarakom Bird Sanctuary,3.8,No
299,Kufri Fun World,3.8,No
300,Magnetic Hill,3.7,No


In [93]:
# масштабирование признаков на основе нормировки и стандартизации

from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()

df_norm["GoogleReviewRatingNorm"] = min_max_scaler.fit_transform(
    df_norm["GoogleReviewRating"].to_numpy().reshape(-1, 1)
).reshape(df_norm["GoogleReviewRating"].shape)

display(df_norm[["Name", "GoogleReviewRating", "GoogleReviewRatingNorm"]])

display()

stndart_scaler = preprocessing.StandardScaler()

df_norm["GoogleReviewRatingNorm"] = stndart_scaler.fit_transform(
    df_norm["GoogleReviewRating"].to_numpy().reshape(-1, 1)
).reshape(df_norm["GoogleReviewRating"].shape)

display(df_norm[["Name", "GoogleReviewRating", "GoogleReviewRatingNorm"]])


,Name,GoogleReviewRating,GoogleReviewRatingNorm
0,Rann Utsav,4.9,1.000000
1,Golden Temple (Harmandir Sahib),4.9,1.000000
2,Pangong Tso,4.9,1.000000
3,Siddhivinayak Temple,4.8,0.971429
4,Somnath Temple,4.8,0.971429
...,...,...,...
297,Kodaikanal Lake,3.9,0.714286
298,Kumarakom Bird Sanctuary,3.8,0.685714
299,Kufri Fun World,3.8,0.685714
300,Magnetic Hill,3.7,0.657143


,Name,GoogleReviewRating,GoogleReviewRatingNorm
0,Rann Utsav,4.9,1.485456
1,Golden Temple (Harmandir Sahib),4.9,1.485456
2,Pangong Tso,4.9,1.485456
3,Siddhivinayak Temple,4.8,1.122799
4,Somnath Temple,4.8,1.122799
...,...,...,...
297,Kodaikanal Lake,3.9,-2.141123
298,Kumarakom Bird Sanctuary,3.8,-2.503781
299,Kufri Fun World,3.8,-2.503781
300,Magnetic Hill,3.7,-2.866439
